In [44]:
import numpy as np
import math
import scipy.linalg as la

# Semana 2 PIBIC

## Gerando Tabelas Verdade (Truth Tables)

Para a geração de tabelas verdade com n inputs no circuito, com $n \ge 1$. Esta é uma função recursiva que gera uma tabela verdade em forma de matriz para n inputs arbitrários.

Talvez existam formas mais rápidas e menores de fazê-las, mas esta função é mais legível para os usuários e para os parceiros da pesquisa. Modifiquei o nome das variáveis para auxiliar no processo.

Além disso, modifiquei-a para que retornasse um numpy array ao invés de um array convencional do python, aumentando a eficiência da função visto que numpy arrays são mais rápidos de se fazer modificações na função e até mesmo posteriores.

In [45]:
def Tv(NumberOfInputs):
    """
    Retorna uma tabela verdade (TruthTable) dado um circuito com n inputs maiores ou iguais a 1
    Checa se a variável NumberOfInputs é inteira. Se sim, continua normalmente, se não lança uma exceção com a mensagem
    'Parameter \'NumberOfInputs\' must be a integer'

    Parâmetros
    ------------------------------------------------
    :param NumberOfInputs: número de inputs do circuito lógico, deve ser um número inteiro (obrigatoriamente)
    
    Retorno
    ------------------------------------------------
    :return: Retorna a tabela verdade como uma lista do Python (caso necessário, transformar o retorno em numpy array)
    """
    
    if not isinstance(NumberOfInputs, int):
        raise ValueError('Parameter \'number_of_inputs\' must be a integer') 
    if NumberOfInputs < 1:
        return [[]]
    
    SubTable = TruthTableGenerator(NumberOfInputs - 1)
    return [row + [v] for row in SubTable for v in [0,1]]

In [46]:
Table = np.array(Tv(3))
Table

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]])

## Modulação PAM

Quando tratamos de tabelas verdade de circuitos eletrônicos, muitas vezes precisamos converter esses sinais em Modulação por Amplitude do Pulso (PAM: Pulse Amplitude Moduling) para realizarmos cálculos ou verificações e depois retornamos a representação do circuito a meneira original, com 0's e 1's.

Para isso, convertemos os campos (labels) das tabelas verdades por $1+\epsilon$ caso o valor no campo seja 1 ou $1-\epsilon$ caso seja 0. A matriz resultante é chamada de $\textbf{Imp}$.

No exemplo a seguir, utilizaremos a variável table definida no código acima para os testes e o valor $\epsilon = 5$:

In [47]:
def PAM(TruthTable, epsilon):
    """
    Função que retorna uma modulação PAM para uma dada tabela verdade (TruthTable) e certo epsilon real arbitrário.
    Não modifica a tabela verdade original, apenas realiza alterações em uma cópia e a retorna no final.
    Funciona para qualquer tabela verdade (truth_table) com n inputs (Matriz não precisa ser quadrada)

    Parâmetros
    ------------------------------------------------
    :param TruthTable: tabela verdade a ser analisada e transformada em modulação PAM
    :param epsilon: epsilon no qual os valores serão calculados
    
    Retorno
    ------------------------------------------------
    :return: retorna um numpy array como matriz da modulação PAM da tabela verdade recebida como parâmetro
    """
    PamTable = TruthTable.copy()
    rows, columns = PamTable.shape
    for row in range(rows):
        for column in range(columns):
            if PamTable[row][column] == 1:
                PamTable[row][column] = 1 + epsilon
            else:
                PamTable[row][column] = 1 - epsilon
    return PamTable

In [48]:
Imp = PAM(Table, 5)
print(f'IMP: \n{Imp}\n\nTABELA VERDADE ORIGINAL: \n{Table}')

IMP: 
[[-4 -4 -4]
 [-4 -4  6]
 [-4  6 -4]
 [-4  6  6]
 [ 6 -4 -4]
 [ 6 -4  6]
 [ 6  6 -4]
 [ 6  6  6]]

TABELA VERDADE ORIGINAL: 
[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]


Note que a linha de código dada por: 

``PAM_table = truth_table.copy()``

faz com que a variável PAM_table seja uma cópia da truth_table, não alterando assim a tabela verdade inicial recebdia como parãmetro. Isso é importante pois podemos realizar os cálculos necessários com a variável Imp que guarda o valor de retorno da função sem perder a tabela verdade original (Desfaz a necessidade da função $\textbf{InvPAM}$).

## Geração de Tabelas Verdade Padrões (OR, AND)

Para dar continuidade à pesquisa, precisamos de uma função que nos retorne tabelas verdade de operadores lógicos padrões (OR, AND).

In [49]:
def OrTruthTableGenerator(TruthTable):
    """
    Retorna uma tabela verdade do circuito lógico OR de acordo com uma tabela verdade (TruthTable). 
    Checa se a tabela está vazia, se sim a retorna (vazia), se não, realiza os cálculos.
    
    Parâmetros
    ------------------------------------------------
    :param TruthTable: tabela verdade a ser analisada
    
    Retorno
    ------------------------------------------------
    :return: Retorna a coluna (numpt array) com os valores da tabela verdade OR dos inputs da tabela verdade recebida 
    como parâmetro
    """
    rows, columns = TruthTable.shape
    ResultTable = np.zeros(shape=(rows, 1), dtype=int)
    
    if TruthTable.size == 0:
        return TruthTable
    
    for row in range(rows):
        BooleanRowArray = TruthTable[row][:]
        if np.any(BooleanRowArray):
            ResultTable[row][0] = 1
        
    return ResultTable

In [50]:
def AndTruthTableGenerator(TruthTable):
    """
    Retorna uma tabela verdade do circuito lógico AND de acordo com uma tabela verdade (TruthTable). 
    Checa se a tabela está vazia, se sim a retorna (vazia), se não, realiza os cálculos.
    
    Parâmetros
    ------------------------------------------------
    :param TruthTable: tabela verdade a ser analisada
    
    Retorno
    ------------------------------------------------
    :return: Retorna a coluna (numpy array) com os valores da tabela verdade AND dos inputs da tabela verdade recebida 
    como parâmetro
    """
    rows, columns = TruthTable.shape
    ResultTable = np.zeros(shape=(rows, 1), dtype=int)
    
    if TruthTable.size == 0:
        return TruthTable
    
    for row in range(rows):
        BooleanRowArray = TruthTable[row][:]
        if np.all(BooleanRowArray):
            ResultTable[row][0] = 1;
    
    return ResultTable

In [51]:
OR = OrTruthTableGenerator(Table)
AND = AndTruthTableGenerator(Table)
print(f'OR TRUTH TABLE: \n{OR} \n\nAND TRUTH TABLE: \n{AND}')

OR TRUTH TABLE: 
[[0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]] 

AND TRUTH TABLE: 
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]]


Perceba que os exemplos utilizados para os cálculos das funções são feitos utilizando tabelas verdade de 3 inputs. No entanto, como já documentado, essas funções aceitam tabelas verdade de n inputs, sendo n um número natural.

## Código de Teste

Os códigos abaixo realizam testes com as funções abordadas, seguindo o modelo (exemplo para o arranjo triangular):

$A = ArranjoT rinagular(α, β, γ) \newline T = MatrizTransferencia(A) \newline Inp = PAM(TV(3)) \newline Out = T Inp^{T} \newline Out = Out^{T} \newline OutPAM = PAM(OR)$

In [52]:
def ArranjoTriangular(alpha, beta, gama):
    A = np.array([[0, complex(0, gama), complex(0, beta)], 
                  [complex(0, gama), 0, complex(0, alpha)], 
                  [complex(0, beta), complex(0, alpha), 0]])
    return A

In [53]:
def MatrizTransferenciaTriangular(A):
    """
    Retorna a matriz de transferência triangular T a partir de uma matriz de arranjo triangular A a partir dos passos já citados
    Erro percentual de 10^(-14) a 10^(-16), ínfimo.
    
    Parâmetros
    --------------------------------------
    :param A: matriz de Arranjo Triangular como numpy array
    
    Retorno
    
    :return:  matriz numpy array de transferência triangular T calculada com erro ínfimo.
    """
    eigvals, eigvecs = la.eig(A)
    D = np.array([[eigvals[0], 0, 0], 
                  [0, eigvals[1], 0], 
                  [0, 0, eigvals[2]]])
    P = eigvecs.copy()
    E = np.array([[np.exp(eigvals[0]), 0, 0], 
                  [0, np.exp(eigvals[1]), 0], 
                  [0, 0, np.exp(eigvals[2])]])
    
    if np.allclose(A, P @ D @ la.inv(P), atol=1e-17):
        raise ValueError(f'Parameter Matrix A has not property A == PDP^(-1) for a tolerance of {1e-17}')
    
    # T é a matriz de transferência
    T = P @ E @ la.inv(P)
    
    return T

In [54]:
A = ArranjoTriangular(1, 2, 3)
T = MatrizTransferenciaTriangular(A)
Inp = PAM(np.array(Tv(3)), 5)
Out = T @ np.transpose(Inp)
Out = np.transpose(Out)
OutPAM = PAM(OR, 5)

In [55]:
print(f'A: \n{A}\n\nT: \n{T}\n\nInp: \n{Inp}\n\nOut: \n{Out}\n\nOutPAM: \n{OutPAM}')

A: 
[[0.+0.j 0.+3.j 0.+2.j]
 [0.+3.j 0.+0.j 0.+1.j]
 [0.+2.j 0.+1.j 0.+0.j]]

T: 
[[-0.78640394-0.33599426j  0.27205028-0.3998916j  -0.03223645-0.18361063j]
 [ 0.27205028-0.3998916j  -0.40191798-0.48536506j -0.59284716+0.13225087j]
 [-0.03223645-0.18361063j -0.59284716+0.13225087j  0.23889194-0.73431637j]]

Inp: 
[[-4 -4 -4]
 [-4 -4  6]
 [-4  6 -4]
 [-4  6  6]
 [ 6 -4 -4]
 [ 6 -4  6]
 [ 6  6 -4]
 [ 6  6  6]]

Out: 
[[ 2.18636044+3.67798595j  2.89085945+3.01202312j  1.54476669+3.1427045j ]
 [ 1.8639959 +1.84187969j -3.03761217+4.33453186j  3.93368613-4.20045924j]
 [ 4.90686327-0.32093003j -1.12832038-1.84162744j -4.38370493+4.46521324j]
 [ 4.58449873-2.15703629j -7.056792  -0.5191187j  -1.99478549-2.8779505j ]
 [-5.67767896+0.31804331j  5.61136228-0.98689286j  1.22240215+1.30659825j]
 [-6.00004349-1.51806295j -0.31710934+0.33561588j  3.61132159-6.03656549j]
 [-2.95717613-3.68087267j  1.59218245-5.84054342j -4.70606947+2.62910698j]
 [-3.27954066-5.51697893j -4.33628917-4.51803468j -2.317